# Get the bounding box of a projected raster that crosses the antimeridian

This notebook demonstrates a hacky method to convert the bounds from a projected raster (demonstrated with a geotiff) that crosses the antimeridian in WGS84 space.

### The issue with an existing method

A typical way to do this in python would be to simply use the `rasterio.warp.transform_bounds` function to transform the bounds of a raster. Here is an example raster:

In [1]:
!gdalinfo data/tas_mean_C_ar5_5ModelAvg_rcp85_01_2006.tif

Driver: GTiff/GeoTIFF
Files: data/tas_mean_C_ar5_5ModelAvg_rcp85_01_2006.tif
Size is 398, 211
Coordinate System is:
PROJCRS["NAD83 / Alaska Albers",
    BASEGEOGCRS["NAD83",
        DATUM["North American Datum 1983",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4269]],
    CONVERSION["Alaska Albers (meters)",
        METHOD["Albers Equal Area",
            ID["EPSG",9822]],
        PARAMETER["Latitude of false origin",50,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8821]],
        PARAMETER["Longitude of false origin",-154,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8822]],
        PARAMETER["Latitude of 1st standard parallel",55,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8823]],
        PARAMETER["Latitude of 2nd standard parallel",65,
       

Using the `transform_bounds` function, we can get to WGS84:

In [2]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from rasterio.warp import transform_bounds


with rio.open("data/tas_mean_C_ar5_5ModelAvg_rcp85_01_2006.tif") as src:
    src_crs = src.crs
    src_bounds = src.bounds

dst_crs = CRS.from_epsg(4326)
print("WSEN:", np.round(transform_bounds(src_crs, dst_crs, *src_bounds), 4))

WSEN: [-179.9402   40.1605  179.3829   82.593 ]


The issue here is that the most western and eastern bounds are simply min / max values on the [-180, 180] scale. We know there are grid cells that are further "west" of the easternmost point, but how do we find out just how far west they go?

### The solution

A quick fix for this is to use a translated WGS84 CRS. Simply replace the prime meridian with the antimeridian when defining the destination CRS:

In [3]:
# replace the relevant parts of the well-known text string
dst_crs = CRS.from_wkt(
    CRS.from_epsg(4326).to_wkt().replace('PRIMEM["Greenwich",0', 'PRIMEM["Greenwich",180')
)

Do the re-projection:

In [4]:
bounds = transform_bounds(src_crs, dst_crs, *src_bounds)
print("WSEN:", np.round(bounds, 4))

WSEN: [-42.1694  40.1605 108.2349  82.593 ]


And translate back:

In [5]:
print("WSEN:", np.round((bounds[0] - 180, bounds[1], bounds[2] - 180, bounds[3]), 4))

WSEN: [-222.1694   40.1605  -71.7651   82.593 ]


Putting this in a function for convenience, using the same imports as above:

In [6]:
def get_wgs84_extent(gtiff_fp):
    with rio.open(gtiff_fp) as src:
        src_crs = src.crs
        src_bounds = src.bounds
    dst_crs = CRS.from_wkt(
        CRS.from_epsg(4326).to_wkt().replace('PRIMEM["Greenwich",0', 'PRIMEM["Greenwich",180')
    )
    bounds = transform_bounds(src_crs, dst_crs, *src_bounds)
    new_bounds = np.round((bounds[0] - 180, bounds[1], bounds[2] - 180, bounds[3]), 4)
    
    return new_bounds


print("WSEN:", get_wgs84_extent("data/tas_mean_C_ar5_5ModelAvg_rcp85_01_2006.tif"))

WSEN: [-222.1694   40.1605  -71.7651   82.593 ]
